In [ ]:
import pandas as pd
import numpy as np
from pandas_datareader import data
import pandas_datareader.data as web
import datetime
import matplotlib.pyplot as plt
import sklearn.model_selection

start=datetime.datetime(2005,1,1)
end=datetime.datetime(2022,10,31)
prices=web.DataReader('^GSPC','yahoo',start,end)

df=prices[['Close', 'Low','Open' ,'High' ,'Volume']]

df['MA60']=df["Close"].rolling(60,min_periods=1).mean()
df['MA20']=df["Close"].rolling(20,min_periods=1).mean()
df['MA10']=df["Close"].rolling(10,min_periods=1).mean()
df['Daily Return'] = df['Close'].pct_change()

df['diff']=df['Daily Return'].shift(-1)
df['up']= df['diff'].apply(lambda x: 2 if x>0.002 else (0 if x<-0.002 else 1 ))
plt.plot(df['Close'],color="red", label='Close')
plt.plot(df['MA60'],color="blue", label='MA60')

fig , ax = plt.subplots(3, 2, sharex=True)
